In [2]:
import os
import openai
from datetime import datetime
from dotenv import load_dotenv()
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [20]:
def create_files(usecase: str, train_file: str, val_file: str=None):
    openai.File.create(
        file=open(train_file, "rb"),
        purpose='fine-tune',
        user_provided_filename="train_"+usecase+str(datetime.now()).replace(" ", "_")
    )

    if val_file is not None:
        openai.File.create(
            file=open(val_file, "rb"),
            purpose='fine-tune',
            user_provided_filename="val_"+usecase+str(datetime.now()).replace(" ", "_")
        )

def create_jobs(train_file: str, val_file: str=None):
    if val_file is None:
        openai.FineTuningJob.create(training_file = train_file, model = "gpt-3.5-turbo")    
    else:
        openai.FineTuningJob.create(training_file = train_file, validation_file = val_file, model = "gpt-3.5-turbo")

In [8]:
usecase = "blotout"
train_file = "/Users/abhinaykumar/Desktop/Work/featherflow/llama-tests/datasets/25dd7090-0b9f-4ffe-aafb-ad3266011955/training/training_data.jsonl"
create_files(usecase=usecase, train_file=train_file)

In [38]:
print(openai.File.list()["data"])

[<File file id=file-ZIPZW8WrCSFsXuK8tMfIXn76 at 0x10654a860> JSON: {
  "object": "file",
  "id": "file-ZIPZW8WrCSFsXuK8tMfIXn76",
  "purpose": "fine-tune",
  "filename": "finetuning_data_prepared_train.jsonl",
  "bytes": 1617332,
  "created_at": 1687371043,
  "status": "processed",
  "status_details": null
}, <File file id=file-qreHIzjsBy7T3cDPLu6SsMok at 0x106599cc0> JSON: {
  "object": "file",
  "id": "file-qreHIzjsBy7T3cDPLu6SsMok",
  "purpose": "fine-tune",
  "filename": "finetuning_data_prepared_train.jsonl",
  "bytes": 1383286,
  "created_at": 1687359847,
  "status": "processed",
  "status_details": null
}, <File file id=file-13gntKbHirnJ8PyTRksKtj59 at 0x106599770> JSON: {
  "object": "file",
  "id": "file-13gntKbHirnJ8PyTRksKtj59",
  "purpose": "fine-tune",
  "filename": "finetuning_data_prepared_train.jsonl",
  "bytes": 1617332,
  "created_at": 1687371816,
  "status": "processed",
  "status_details": null
}, <File file id=file-Tw8R8WJmiG1j76lRFAAgszX1 at 0x1065999a0> JSON: {
 

train_blotout2023-10-11_15:05:47.694317 : file-RUidKpmKabosRs3PF81YhGes

In [21]:
create_jobs("file-RUidKpmKabosRs3PF81YhGes")

In [39]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x10658c7c0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-VnAhr8GL7242CeSXHXBQPHOn",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1697025077,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-Ts7Yq2E7boGzRTMgBIbmsUmj",
      "result_files": [],
      "status": "validating_files",
      "validation_file": null,
      "training_file": "file-RUidKpmKabosRs3PF81YhGes",
      "hyperparameters": {
        "n_epochs": "auto"
      },
      "trained_tokens": null,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-EBJFa5rJIYkzr8cCAgV5pX6I",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1697017093,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-Ts7Yq2E7boGzRTMgBIbmsUmj",
      "result_files": [],
      "status": "cancelled",
      "validation_file": null,
      "training_file": "fi

finetune job id : ftjob-VnAhr8GL7242CeSXHXBQPHOn

In [40]:
finetune_job = "ftjob-VnAhr8GL7242CeSXHXBQPHOn"

In [41]:
event_list = openai.FineTuningJob.list_events(id=finetune_job)["data"]
# print(event_list[:3])
training_metrics = [(event["data"]["step"], event["data"]["train_loss"], event["data"]["train_mean_token_accuracy"]) for event in event_list if event["type"]=="metrics"]

In [42]:
openai.FineTuningJob.list_events(id=finetune_job, limit=10)

<OpenAIObject list at 0x10654f1d0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-Ct0o85kM2rZv9BRGrDH6o0Gg",
      "created_at": 1697025077,
      "level": "info",
      "message": "Validating training file: file-RUidKpmKabosRs3PF81YhGes",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-n6l8xD30Lt45y2IXWlfWHPDu",
      "created_at": 1697025077,
      "level": "info",
      "message": "Created fine-tuning job: ftjob-VnAhr8GL7242CeSXHXBQPHOn",
      "data": {},
      "type": "message"
    }
  ],
  "has_more": false
}

In [43]:
training_metrics

[]

# Finetuned Model

ft:gpt-3.5-turbo-0613:saydata::7wPg9WGE

In [11]:
# Testing it out
import json

with open("metadata/train/small_finetune_test_dataset.jsonl", "r") as f:
    test_data = list(f)[:100]

run_prompts = []

for prompt_json in test_data:
    prompt = json.loads(prompt_json)
    temp = {
        "messages": [i for i in prompt["messages"] if i["role"]!="assistant"]
    }
    true_value = [i for i in prompt["messages"] if i["role"]=="assistant"][0]
    run_prompts.append((temp, true_value))

In [12]:
run_prompts[0]

({'messages': [{'role': 'system',
    'content': 'You are an SQL chatbot. Generate relevant MySQL query for questions'},
   {'role': 'user',
    'content': 'Generate the MySQL query to answer this question: How many properties have undergone renovations?\n\n                Use this database metadata to generate the query: \n\n                [{\'table\': \'maintenance_logs\', \'table_description\': "The \'maintenance_logs\' table in the \'real_estate_mgmt\' database stores records of property maintenance activities. It includes details such as the log ID, property ID, maintenance date and type, description, contractor information, cost, invoice number, payment status and date, next maintenance date, and timestamps for record creation and updates.", \'columns\': {\'property_id\': (\'The `property_id` refers to the unique identifier of the property associated with the maintenance log.\', [101, 102, 103, 104, 105, 106, 107, 108, 109, 110]), \'maintenance_type\': (\'The `maintenance_type` 

In [ ]:
def wrapper_llm()

In [14]:
from tqdm import tqdm
from joblib import Parallel, delayed
from generate_metadata import LLM
llm_finetuned = LLM(model="ft:gpt-3.5-turbo-0613:saydata::7wPg9WGE")

finetuned_results = Parallel(n_jobs=4)(delayed(llm_finetuned.create)(prompt[0]["messages"]) for prompt in tqdm(run_prompts))



















































100%|██████████| 100/100 [01:32<00:00,  1.08it/s]


In [15]:
llm_vanilla = LLM(model="gpt-3.5-turbo-16k")

vanilla_results = Parallel(n_jobs=4)(delayed(llm_vanilla.create)(prompt[0]["messages"]) for prompt in tqdm(run_prompts))





 30%|███       | 167/550 [24:28<56:07,  8.79s/it]














































100%|██████████| 100/100 [02:26<00:00,  1.47s/it]
